## Hangman 

By Matthew O'Malley-Nichols

### Implementation

#### Hyperparameters

In [1]:
import pandas as pd
from time import time

In [2]:
file_location = './hw1_word_counts_05-1.txt' 
num_positions = 5 

#### Class Definition

In [3]:
class Hangman:
    def __init__(self, file_location, num_positions): 
        self.df = pd.read_csv(file_location, sep=' ', header=None, names=['word','freq','prior','posterior', 'likelihood'])
        self.num_positions = num_positions
        self.df['word'] = self.df['word'].map(lambda x: x.lower())
        assert all([len(word) == num_positions for word in self.df['word']])
        priors = self.compute_prior_probabilities()
        self.df['prior'] = self.compute_prior_probabilities()
        self.df['posterior'] = self.df['prior']
        self.df['likelihood'] = 1
        self.evidence = {'correct': [None for i in range(num_positions)], 'incorrect': set()}

    def get_words(self):
        return self.df['word']

    def get_priors(self):
        return self.df['prior'] 

    def set_priors(self, priors):
        self.df['prior'] = priors

    def get_posteriors(self):
        return self.df['posterior']

    def set_posteriors(self, posteriors):
        self.df['posterior'] = posteriors

    def get_likelihoods(self):
        return self.df['likelihood']

    def set_likelihoods(self, likelihoods):
        self.df['likelihood'] = likelihoods

    def get_num_positions(self):
        return self.num_positions
        
    def reset(self):
        self.df['posterior'] = self.df['prior']
        self.df['likelihood'] = 1
        self.evidence = {'correct': [None for i in range(self.num_positions)], 'incorrect': set()}

#### Compute Prior Probabilities
$P(W=w) = \frac{count(w)}{\sum_{w'}count(w')}$

In [4]:
def compute_prior_probabilities(self):
    total_count = self.df['freq'].sum()
    priors = self.df['freq'] / total_count
    assert sum(priors) == 1
    return priors 
setattr(Hangman, 'compute_prior_probabilities', compute_prior_probabilities)

#### Compute Likelihoods

$P(E | W = w) = 
\begin{cases} 
0 & \text{if } E_i \not \implies w_i \forall E_i\\
1 & \text{otherwise}
\end{cases}
$


In [5]:
def get_word_likelihood(self, evidence, word, num_positions):
    for i in range(num_positions):
        if word[i] in evidence['incorrect']:
            return 0
        if evidence['correct'][i] is not None:
            if word[i] != evidence['correct'][i]:
                return 0
    for letter in evidence['correct']:
        if letter is None:
            continue
        if word.count(letter) > evidence['correct'].count(letter):
            return 0
    return 1

def compute_word_likelihoods(self, evidence, words, num_positions):
    # Assumes lowercase
    self.evidence = evidence
    likelihoods = words.map(lambda x: self.get_word_likelihood(evidence, x, num_positions)) 
    return likelihoods

setattr(Hangman, 'get_word_likelihood', get_word_likelihood)
setattr(Hangman, 'compute_word_likelihoods', compute_word_likelihoods)

#### Compute Posterior Probabilities
$P(W=w | E) = \frac{P(E|W=w)P(W=w)}{
\sum_{w'}P(E|W=w')P(W=w')
}$

In [6]:
def compute_posterior_probabilities(self, priors, word_likelihoods):
    # self.df
    unnormalized_posteriors = priors * word_likelihoods
    posteriors = (unnormalized_posteriors / sum(unnormalized_posteriors))
    return posteriors
setattr(Hangman, 'compute_posterior_probabilities', compute_posterior_probabilities)

#### Compute Predictive Probabilities
$P(L_i = l \text{ for } i \in \{1,2,3,4,5\} | E) = \sum_{w}P(L_i = l \text{ for } i \in \{1,2,3,4,5\} | W=w)P(W=w|E)$

In [7]:
def get_letter_likelihood(self, guess, word, guessable_indices, evidence):
    if guess in evidence['incorrect'] or guess in evidence['correct']:
        return 0
    for i in guessable_indices:
        if word[i] == guess:
            return 1
    return 0

def compute_predictive_probabilities(self, guess, posteriors, words, evidence, num_positions):
    considered_words = posteriors > 0
    filtered_words = words[considered_words]
    guessable_indices = [i for i in range(num_positions) if evidence['correct'][i] is None]
    letter_likelihoods = filtered_words.map(lambda word: self.get_letter_likelihood(guess, word, guessable_indices, evidence))    
    predictives = sum(letter_likelihoods * posteriors[considered_words])
    return predictives  
    
setattr(Hangman, 'compute_predictive_probabilities', compute_predictive_probabilities)
setattr(Hangman, 'get_letter_likelihood', get_letter_likelihood)

#### Inference

In [8]:
def inference(self, evidence):
    guesses = list('abcdefghijklmnopqrstuvwxyz')
    predictives = list()
    word_likelihoods = self.compute_word_likelihoods(evidence, self.get_words(), self.get_num_positions())
    posteriors = self.compute_posterior_probabilities(self.get_priors(), word_likelihoods)
    for letter in guesses:
        predictive = self.compute_predictive_probabilities(letter, posteriors, self.get_words(), evidence, self.get_num_positions())
        predictive = round(predictive,4)
        predictives.append(predictive)
    results = pd.DataFrame({'letter': guesses, 'predictive': predictives})
    return results

setattr(Hangman, 'inference', inference)

#### Results checking

In [9]:
def print_most_least_frequent_words(self):
    print(f"Fifteen most frequent words: \n{self.df.sort_values(by='freq', ascending=False)[:15]}")
    print(f"Fifteen least frequent words: \n{self.df.sort_values(by='freq', ascending=True)[:15]}")
setattr(Hangman, 'print_most_least_frequent_words', print_most_least_frequent_words)

In [10]:
methods = [meth for meth in dir(Hangman) if not meth.startswith('__')] 
print(f"Methods added: {methods}")

Methods added: ['compute_posterior_probabilities', 'compute_predictive_probabilities', 'compute_prior_probabilities', 'compute_word_likelihoods', 'get_letter_likelihood', 'get_likelihoods', 'get_num_positions', 'get_posteriors', 'get_priors', 'get_word_likelihood', 'get_words', 'inference', 'print_most_least_frequent_words', 'reset', 'set_likelihoods', 'set_posteriors', 'set_priors']


### Results

#### Priors Initialization

In [11]:
start = time()
hangman = Hangman(file_location, num_positions)
end = time()
print(f"Time to load data: {(end - start) * 1000} ms")

Time to load data: 17.34304428100586 ms


In [12]:
hangman.print_most_least_frequent_words()

Fifteen most frequent words: 
       word    freq     prior  posterior  likelihood
5821  three  273077  0.035627   0.035627           1
5102  seven  178842  0.023333   0.023333           1
1684  eight  165764  0.021626   0.021626           1
6403  would  159875  0.020858   0.020858           1
18    about  157448  0.020542   0.020542           1
5804  their  145434  0.018974   0.018974           1
6320  which  142146  0.018545   0.018545           1
73    after  110102  0.014365   0.014365           1
1975  first  109957  0.014346   0.014346           1
1947  fifty  106869  0.013943   0.013943           1
4158  other  106052  0.013836   0.013836           1
2073  forty   94951  0.012388   0.012388           1
6457  years   88900  0.011598   0.011598           1
5806  there   86502  0.011286   0.011286           1
5250  sixty   73086  0.009535   0.009535           1
Fifteen least frequent words: 
       word  freq         prior     posterior  likelihood
712   bosak     6  7.827935e-07  

#### Setup Hangman Games

In [13]:
num_games = 9

In [14]:
correct_guesses = [
    [None,None,None,None,None]
    for i in range(num_games)
]
correct_guesses[2][0] = 'a'
correct_guesses[3][0] = 'a'
correct_guesses[2][4] = 's'
correct_guesses[3][4] = 's'
correct_guesses[4][2] = 'o'
correct_guesses[6][0] = 'd'
correct_guesses[7][0] = 'd'
correct_guesses[6][3] = 'i'
correct_guesses[7][3] = 'i'
correct_guesses[8][1] = 'u'

for correct in correct_guesses:
    print(correct)

[None, None, None, None, None]
[None, None, None, None, None]
['a', None, None, None, 's']
['a', None, None, None, 's']
[None, None, 'o', None, None]
[None, None, None, None, None]
['d', None, None, 'i', None]
['d', None, None, 'i', None]
[None, 'u', None, None, None]


In [15]:
incorrect_guesses = [
    set()
    for i in range(num_games)
]
incorrect_guesses[1].update(['e', 'a'])
incorrect_guesses[3].add('i')
incorrect_guesses[4].update(['a', 'e', 'm', 'n', 't'])
incorrect_guesses[5].update(['e', 'o'])
incorrect_guesses[7].add('a')
incorrect_guesses[8].update(['a', 'e', 'i', 'o', 's'])

for incorrect in incorrect_guesses:
    print(incorrect)

set()
{'e', 'a'}
set()
{'i'}
{'e', 'm', 'a', 't', 'n'}
{'o', 'e'}
set()
{'a'}
{'o', 'e', 's', 'i', 'a'}


#### Play Hangman

In [16]:
best_guesses = []
for i in range(num_games):
    hangman.reset()
    evidence = {
        'correct': correct_guesses[i],
        'incorrect': incorrect_guesses[i]
    }
    results = hangman.inference(evidence) 
    best_guess = results.sort_values(by='predictive', ascending=False).iloc[0].to_dict()
    best_guesses += [best_guess]
    
hangman_results = pd.DataFrame({
    'correct': correct_guesses,
    'incorrect': incorrect_guesses,
    'best_guess': [bg['letter'] for bg in best_guesses],
    'predictive': [bg['predictive'] for bg in best_guesses]
})
print(hangman_results)

                          correct        incorrect best_guess  predictive
0  [None, None, None, None, None]               {}          e      0.5394
1  [None, None, None, None, None]           {e, a}          o      0.5340
2        [a, None, None, None, s]               {}          e      0.7715
3        [a, None, None, None, s]              {i}          e      0.7127
4     [None, None, o, None, None]  {e, m, a, t, n}          r      0.7454
5  [None, None, None, None, None]           {o, e}          i      0.6366
6        [d, None, None, i, None]               {}          a      0.8207
7        [d, None, None, i, None]              {a}          e      0.7521
8     [None, u, None, None, None]  {o, e, s, i, a}          y      0.6270
